Done! Congratulations on your new bot. You will find it at t.me/ArbolitoV2_bot. You can now add a description, about section and profile picture for your bot, see /help for a list of commands. By the way, when you've finished creating your cool bot, ping our Bot Support if you want a better username for it. Just make sure the bot is fully operational before you do this.

Use this token to access the HTTP API:
7933607614:AAE68SZk1Ia_1NP8l6-y9PsT8NhuN_Wzvdg
Keep your token secure and store it safely, it can be used by anyone to control your bot.

For a description of the Bot API, see this page: https://core.telegram.org/bots/api

In [1]:
#!pip uninstall python-telegram-bot

In [2]:
#Importamos Librerias
from telegram.ext import Application, CommandHandler, Updater
import logging  

In [3]:
#Importamos el Token
bot_token = "7933607614:AAE68SZk1Ia_1NP8l6-y9PsT8NhuN_Wzvdg"

In [5]:
import logging
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ConversationHandler
import pandas as pd
import io
import asyncio
import nest_asyncio
import os
import importlib.util

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Configure logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

# Define conversation states
CHOOSING_AGENT = 0

# Get the directory of the current script
try:
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    current_dir = os.getcwd()
    
    BASE_DIR = os.path.dirname(current_dir)  # Use current working directory if __file__ is not defined

# Define your command functions
async def start(update, context):
    await update.message.reply_text('Bienvenido/a! Por favor, elige un agente para iniciar el proceso (ej., "BNA"):')
    return CHOOSING_AGENT

async def process_agent(update, context):
    agent = update.message.text.upper()
    script_path = os.path.join(BASE_DIR, 'BNA', 'obtener_cotizacion.py')

    logging.info(f"Attempting to run script for agent {agent}")
    logging.info(f"Looking for script at path: {script_path}")

    if not os.path.exists(script_path):
        await update.message.reply_text(f"Lo siento, no se encontró ningún agente llamado {agent} en {script_path}.")
        return ConversationHandler.END

    # Send an intermediate message
    wait_message = await update.message.reply_text(f"Por favor espere, estoy procesando la solicitud para {agent}. Esto puede tomar unos segundos...")

    try:
        # Import and run the script
        spec = importlib.util.spec_from_file_location("module.name", script_path)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)

        # Assuming the script has a get_exchange_rate function
        result = module.get_exchange_rate()

        if result:
            # Read the CSV file
            csv_path = os.path.join( 'exchange_rates.csv')
            if not os.path.exists(csv_path):
                await update.message.reply_text(f"Error: CSV file not found at {csv_path}")
                return ConversationHandler.END

            df = pd.read_csv(csv_path)
            
            # Convert DataFrame to CSV string
            csv_buffer = io.StringIO()
            df.to_csv(csv_buffer, index=False)
            csv_buffer.seek(0)

            # Delete the wait message
            await wait_message.delete()

            # Send the CSV file
            await context.bot.send_document(
                chat_id=update.effective_chat.id,
                document=io.BytesIO(csv_buffer.getvalue().encode()),
                filename='exchange_rates.csv',
                caption=f'Aquí están los "exchange rates" para {agent}.'
            )
        else:
            # Delete the wait message
            await wait_message.delete()
            await update.message.reply_text(f"El script para {agent} se ejecutó, pero no devolvió ningún resultado.")

    except Exception as e:
        logging.error(f"Error running script for {agent}: {str(e)}", exc_info=True)
        # Delete the wait message
        await wait_message.delete()
        await update.message.reply_text(f"Ocurrió un error al procesar la solicitud para {agent}: {str(e)}")

    return ConversationHandler.END

async def cancel(update, context):
    await update.message.reply_text('Operation cancelled.')
    return ConversationHandler.END

async def main():
    # Create the application and pass your bot's token
    application = Application.builder().token(bot_token).build()

    # Add conversation handler
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            CHOOSING_AGENT: [MessageHandler(filters.TEXT & ~filters.COMMAND, process_agent)]
        },
        fallbacks=[CommandHandler('cancel', cancel)]
    )

    application.add_handler(conv_handler)

    return application

if __name__ == '__main__':
    # Get the current event loop
    loop = asyncio.get_event_loop()

    # Create and configure the application
    application = loop.run_until_complete(main())

    # Start the bot
    try:
        print("Bot started. Press Ctrl+C to stop.")
        loop.run_until_complete(application.run_polling())
    except KeyboardInterrupt:
        print("Bot stopped.")
    finally:
        loop.run_until_complete(application.shutdown())

Bot started. Press Ctrl+C to stop.


2025-04-24 16:06:41,017 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7933607614:AAE68SZk1Ia_1NP8l6-y9PsT8NhuN_Wzvdg/getMe "HTTP/1.1 200 OK"
2025-04-24 16:06:41,243 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7933607614:AAE68SZk1Ia_1NP8l6-y9PsT8NhuN_Wzvdg/deleteWebhook "HTTP/1.1 200 OK"
2025-04-24 16:06:41,246 - telegram.ext.Application - INFO - Application started
2025-04-24 16:06:41,950 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7933607614:AAE68SZk1Ia_1NP8l6-y9PsT8NhuN_Wzvdg/getUpdates "HTTP/1.1 200 OK"
2025-04-24 16:06:42,304 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7933607614:AAE68SZk1Ia_1NP8l6-y9PsT8NhuN_Wzvdg/sendMessage "HTTP/1.1 200 OK"
2025-04-24 16:06:52,191 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7933607614:AAE68SZk1Ia_1NP8l6-y9PsT8NhuN_Wzvdg/getUpdates "HTTP/1.1 200 OK"
2025-04-24 16:06:52,424 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7933607614:AA

RuntimeError: Cannot close a running event loop

In [5]:
import os
import logging
import importlib.util
import pandas as pd
import io
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ConversationHandler

# Set up logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the base directory where your script is located
BASE_DIR = r"C:\Users\s1093678\OneDrive - Syngenta\Documents\Crop Protection\Cursos\Master en Data Science\20 - Web Mining\arbolito\BNA"

# Define conversation states
CHOOSING = 0

async def start(update: Update, context):
    await update.message.reply_text(
        "Bienvenido al bot de cotizaciones de bancos. "
        "Por favor, elija un banco (BNA, PROVINCIA, CIUDAD) o escriba 'TODOS' para obtener todas las cotizaciones."
    )
    return CHOOSING

async def process_bank(update: Update, context):
    bank = update.message.text.upper()
    script_path = os.path.join(BASE_DIR, 'obtener_cotizacion_v2.py')

    logging.info(f"Attempting to run script for bank {bank}")
    logging.info(f"Looking for script at path: {script_path}")

    if not os.path.exists(script_path):
        await update.message.reply_text(f"Lo siento, no se encontró el script en {script_path}.")
        return ConversationHandler.END

    # Send an intermediate message
    wait_message = await update.message.reply_text(f"Por favor espere, estoy procesando la solicitud para {bank}. Esto puede tomar unos segundos...")

    try:
        # Import and run the script
        spec = importlib.util.spec_from_file_location("module.name", script_path)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)

        # Call the appropriate function based on the bank
        if bank == 'BNA':
            result = module.get_exchange_rate_BNA()
        elif bank == 'PROVINCIA':
            result = module.get_exchange_rate_banco_provincia()
        elif bank == 'CIUDAD':
            result = module.get_exchange_rate_bancociudad()
        elif bank == 'TODOS':
            result = (module.get_exchange_rate_BNA() and 
                      module.get_exchange_rate_banco_provincia() and 
                      module.get_exchange_rate_bancociudad())
        else:
            await update.message.reply_text(f"Banco no reconocido: {bank}")
            return ConversationHandler.END

        # Check if the CSV file was created
        csv_path = os.path.join(BASE_DIR, 'exchange_rates.csv')
        if not os.path.exists(csv_path):
            logging.error(f"CSV file not created at {csv_path}")
            await update.message.reply_text(f"Error: No se pudo crear el archivo CSV en {csv_path}")
            return ConversationHandler.END

        # Read the CSV file
        df = pd.read_csv(csv_path)
        
        # Convert DataFrame to CSV string
        csv_buffer = io.StringIO()
        df.to_csv(csv_buffer, index=False)
        csv_buffer.seek(0)

        # Delete the wait message
        await context.bot.delete_message(chat_id=update.effective_chat.id, message_id=wait_message.message_id)

        # Send the CSV file
        await context.bot.send_document(
            chat_id=update.effective_chat.id,
            document=io.BytesIO(csv_buffer.getvalue().encode()),
            filename='exchange_rates.csv',
            caption=f'Aquí están los "exchange rates" para {bank}.'
        )

    except Exception as e:
        logging.error(f"Error running script for {bank}: {str(e)}", exc_info=True)
        # Delete the wait message
        await context.bot.delete_message(chat_id=update.effective_chat.id, message_id=wait_message.message_id)
        await update.message.reply_text(f"Ocurrió un error al procesar la solicitud para {bank}: {str(e)}")

    return ConversationHandler.END

def main():
    # Replace 'YOUR_BOT_TOKEN' with your actual bot token
    application = Application.builder().token('7933607614:AAE68SZk1Ia_1NP8l6-y9PsT8NhuN_Wzvdg').build()

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            CHOOSING: [MessageHandler(filters.TEXT & ~filters.COMMAND, process_bank)],
        },
        fallbacks=[],
    )

    application.add_handler(conv_handler)

    # Start the Bot
    application.run_polling()

if __name__ == '__main__':
    main()

RuntimeError: Cannot close a running event loop